<a href="https://colab.research.google.com/github/sank29mane/lays-bio-summery/blob/main/Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade tensorflow

In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset, load_metric
import pandas as pd
import json

In [ ]:
eLife_train = r'/content/drive/MyDrive/Dissertation /train/eLife_train.jsonl'
eLife_val = r'/content/drive/MyDrive/Dissertation /val/eLife_val.jsonl'

In [ ]:
# Create Dataset objects
train_dataset = load_dataset("json", data_files=eLife_train)
val_dataset = load_dataset("json", data_files=eLife_val)

In [ ]:
print(train_dataset)
print(val_dataset)

DatasetDict({
    train: Dataset({
        features: ['lay_summary', 'article', 'headings', 'keywords', 'id'],
        num_rows: 4346
    })
})
DatasetDict({
    train: Dataset({
        features: ['lay_summary', 'article', 'headings', 'keywords', 'id'],
        num_rows: 241
    })
})


In [ ]:
!pip install sacremoses

In [ ]:
!pip install transformers[torch]

In [ ]:
val_dataset=val_dataset.remove_columns(["id","headings","keywords"])

In [ ]:
sample = val_dataset['train'][0]
input = sample['article'][:1024]


In [ ]:
!pip install rouge_score

In [ ]:
#Rouge score (Relevance Scores)
from datasets import load_metric
def compute_rouge(pred, ref=sample['lay_summary']):

  rouge = load_metric("rouge")
  ref = sample['lay_summary']
  results =rouge.compute(predictions=[pred], references=[ref])

  df = pd.DataFrame({
    'Metric': ['rouge1', 'rouge2', 'rougeL','rougeLsum'],
    'Precision': [results[key].mid.precision for key in results],
    'Recall': [results[key].mid.recall for key in results],
    'FMeasure': [results[key].mid.fmeasure for key in results],
  })

  print(df)


In [ ]:
!pip install bert_score

In [ ]:
#Bert score(Relevance Scores)
import pandas as pd
from statistics import mean
def compute_bertscore(pred, ref=['lay_summary']):
    ref = []
    ref.append(sample['lay_summary'])
    bertscore = load_metric("bertscore")
    results = bertscore.compute(predictions=pred, references=ref, lang="en")
    df = pd.DataFrame({
        'Metric': ['precision', 'recall', 'f1'],
        'Score': [mean(results[key]) for key in ['precision', 'recall', 'f1']],
    })

    print(df)

In [ ]:
!pip install textstat

In [ ]:
#FKGL and DRSC scores (Readibility Scores)
import textstat
def calculate_readability_scores(sum):

  if 'summary_text' in sum[0]:
    key = 'summary_text'
  elif 'generated_text' in sum[0]:
    key = 'generated_text'

  text = sum[0][key]
  print(text)
  fkgl = textstat.flesch_kincaid_grade(text)
  dcrs = textstat.dale_chall_readability_score(text)
  print(f"Flesch-Kincaid Grade Level: {fkgl}")
  print(f"Dale-Chall Readability Score: {dcrs}")

In [ ]:
!pip install summac

  Using cached huggingface_hub-0.17.0-py3-none-any.whl (294 kB)
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.1
    Uninstalling huggingface-hub-0.20.1:
      Successfully uninstalled huggingface-hub-0.20.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 2.16.1 requires huggingface-hub>=0.19.4, but you have huggingface-hub 0.17.0 which is incompatible.


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
#Summac (Factuality/consistency Scores)
from summac.model_summac import SummaCZS, SummaCConv
def compute_summac(pred, input=input):
  if 'summary_text' in sum[0]:
    key = 'summary_text'
  elif 'generated_text' in sum[0]:
    key = 'generated_text'

  summary = sum[0][key]
  document = input

  #model_zs = SummaCZS(granularity="sentence", model_name="vitc", device="cpu")
  model_conv = SummaCConv(models=["vitc"], bins='percentile', granularity="sentence", nli_labels="e", device="cpu", start_file="default", agg="mean")

  #score_zs1 = model_zs.score([document], [summary])
  score_conv1 = model_conv.score([document], [summary])
  print("[Summary] SummacConv score: %.3f" % ( score_conv1["scores"][0]))
  #print("[Summary] SummaCZS Score: %.3f; SummacConv score: %.3f" % (score_zs1["scores"][0], score_conv1["scores"][0]))

In [ ]:
from transformers import pipeline
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
# Load the trained model and tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/Dissertation/modelfT5")
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/Dissertation/tokenizerfT5")

# Create the summarization pipeline
summarizer = pipeline("summarization", model=model, tokenizer=tokenizer)
sum = summarizer(input, max_length=250, min_length=80, do_sample=False)

compute_rouge(sum)
compute_bertscore(sum)
calculate_readability_scores(sum)
compute_summac(sum)

<ipython-input-12-f100f6e456a4>:5: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge = load_metric("rouge")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


      Metric  Precision    Recall  FMeasure
0     rouge1   0.312500  0.043228  0.075949
1     rouge2   0.021277  0.002890  0.005089
2     rougeL   0.208333  0.028818  0.050633
3  rougeLsum   0.208333  0.028818  0.050633


/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for bertscore contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/bertscore/bertscore.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.


config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

...

spiece.model:   0%|          | 0.00/760k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/235M [00:00<?, ?B/s]

[Summary] SummacConv score: 0.408


/usr/local/lib/python3.10/dist-packages/summac/model_summac.py:298: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  histograms = torch.FloatTensor(histograms).to(self.device)


In [ ]:
from transformers import pipeline
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
# Load the trained model and tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/Dissertation/modelfT5")
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/Dissertation/tokenizerfT5")

# Initialize the pipeline
text_generator = pipeline("text2text-generation", model=model, tokenizer=tokenizer, device=-1)

# Use the pipeline to generate text
prompt =  'summarize in simple terms:'+input
sum = text_generator(prompt, max_length=250, min_length=200, do_sample=True, temperature=0.7)

print(sum)
compute_rouge(sum)
compute_bertscore(sum)
calculate_readability_scores(sum)
compute_summac(sum)

[{'generated_text': 'Genes inhibit cell fate . GLP-1/Notch signaling is a mechanism by which cell-fate reprograming occurs . Some genes are conserved , and some are mutated . Interestingly , GLP-1/Notch may be a key target for the process . These findings will influence how scientists view stem cells , cellular populations , and the role of stem cells in development . GLP-1/Notch signals a gene called UTX-1 to allow the cell to repair itself without destroying its own cells . This gene is a key mediator of cell fate , but it is also a key regulator of cell fate . This gene is a key regulator of cell fate , which is essential for the formation of a new cell . The GLP-1/Notch pathway is responsible for cellular regeneration , such as reproduction . It also enables the cell to divide into new cells . This process is important for the human body .'}]


/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


      Metric  Precision    Recall  FMeasure
0     rouge1   0.472973  0.201729  0.282828
1     rouge2   0.040816  0.017341  0.024341
2     rougeL   0.216216  0.092219  0.129293
3  rougeLsum   0.216216  0.092219  0.129293


/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for bertscore contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/bertscore/bertscore.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to

      Metric     Score
0  precision  0.834597
1     recall  0.816531
2         f1  0.825465
Genes inhibit cell fate . GLP-1/Notch signaling is a mechanism by which cell-fate reprograming occurs . Some genes are conserved , and some are mutated . Interestingly , GLP-1/Notch may be a key target for the process . These findings will influence how scientists view stem cells , cellular populations , and the role of stem cells in development . GLP-1/Notch signals a gene called UTX-1 to allow the cell to repair itself without destroying its own cells . This gene is a key mediator of cell fate , but it is also a key regulator of cell fate . This gene is a key regulator of cell fate , which is essential for the formation of a new cell . The GLP-1/Notch pathway is responsible for cellular regeneration , such as reproduction . It also enables the cell to divide into new cells . This process is important for the human body .
Flesch-Kincaid Grade Level: 6.9
Dale-Chall Readability Score: 8.66
<All k

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.


[Summary] SummacConv score: 0.225


In [ ]:
from transformers import pipeline
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
# Load the trained model and tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/Dissertation1/model_ft5b")
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/Dissertation1/tokenizer_ft5b")

# Initialize the pipeline
text_generator = pipeline("text2text-generation", model=model, tokenizer=tokenizer)

# Use the pipeline to generate text
prompt =  'Summarise the following article in simple terms:'+ input
sum = text_generator(prompt, max_length=250, min_length=180, do_sample=True, temperature=0.7)

print(sum)
compute_rouge(sum)
compute_bertscore(sum)
calculate_readability_scores(sum)
compute_summac(sum)

[{'generated_text': 'GLP-1/Notch plays a crucial role in cell-fate reprograming , and is an independent regulator of regeneration in C. elegans. Further studies confirm this role by identifying the conserved histone demethylase UTX-1 as a crucial GLP-1/Notch target . These findings provide important information for future research . GLP-1/Notch is a highly soluble protein that mediates cell transformation from undifferentiated germ cells to somatic cells . It is important to identify a crucial role for GLP-1/Notch in the regeneration of germ cells . Overall , this result implies that GLP-1/Notch plays a key role in cell-fate regeneration in C. elegans . These findings suggest that GLP-1/Notch is an important mediator of cell-fate regeneration in C. elegans .'}]


/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


      Metric  Precision    Recall  FMeasure
0     rouge1   0.456000  0.164265  0.241525
1     rouge2   0.056452  0.020231  0.029787
2     rougeL   0.232000  0.083573  0.122881
3  rougeLsum   0.232000  0.083573  0.122881


/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for bertscore contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/bertscore/bertscore.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to

      Metric     Score
0  precision  0.816374
1     recall  0.810047
2         f1  0.813198
GLP-1/Notch plays a crucial role in cell-fate reprograming , and is an independent regulator of regeneration in C. elegans. Further studies confirm this role by identifying the conserved histone demethylase UTX-1 as a crucial GLP-1/Notch target . These findings provide important information for future research . GLP-1/Notch is a highly soluble protein that mediates cell transformation from undifferentiated germ cells to somatic cells . It is important to identify a crucial role for GLP-1/Notch in the regeneration of germ cells . Overall , this result implies that GLP-1/Notch plays a key role in cell-fate regeneration in C. elegans . These findings suggest that GLP-1/Notch is an important mediator of cell-fate regeneration in C. elegans .
Flesch-Kincaid Grade Level: 11.6
Dale-Chall Readability Score: 10.45
<All keys matched successfully>


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.


[Summary] SummacConv score: 0.220


In [ ]:
from transformers import pipeline
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
# Load the trained model and tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/Dissertation1/model_ft5b2")
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/Dissertation1/tokenizer_ft5b2")

# Initialize the pipeline
text_generator = pipeline("text2text-generation", model=model, tokenizer=tokenizer)

# Use the pipeline to generate text
prompt =  'summarize in simple terms:'+ input
sum = text_generator(prompt, max_length=250, min_length=180, do_sample=True, temperature=0.7)

print(sum)
compute_rouge(sum)
compute_bertscore(sum)
calculate_readability_scores(sum)
compute_summac(sum)

[{'generated_text': 'GLP-1/Notch promotes cell-fate replication by activating genes and silenced histone demethylase UTX-1. This new discovery has implications for human development . The GLP-1/Notch signaling pathway in C. elegans germline is required to foster cell-fate reproduction . The new findings highlight the importance of GLP-1/Notch signaling to promote cell-fate reproduced germ cells and somatic cells . Our results will provide the basis for a novel development strategy in the cellular reproduction of germ cells . . . . Reprogramming is at the heart of development . . . However , very little is known about the molecular mechanisms promoting or inhibiting cell-fate reproduction in intact organisms . This is surprising , since GLP-1/Notch is best known for maintaining undifferentiated germline stem cells/progenitors . We propose that GLP-1/Notch promotes reprograming by activating specific genes , silenced by the Polycomb repressive complex 2 ( PRC2 ) and identify the conserve

/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


      Metric  Precision    Recall  FMeasure
0     rouge1   0.398649  0.170029  0.238384
1     rouge2   0.020408  0.008671  0.012170
2     rougeL   0.182432  0.077810  0.109091
3  rougeLsum   0.182432  0.077810  0.109091


/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for bertscore contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/bertscore/bertscore.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to

      Metric     Score
0  precision  0.814373
1     recall  0.818196
2         f1  0.816280
GLP-1/Notch promotes cell-fate replication by activating genes and silenced histone demethylase UTX-1. This new discovery has implications for human development . The GLP-1/Notch signaling pathway in C. elegans germline is required to foster cell-fate reproduction . The new findings highlight the importance of GLP-1/Notch signaling to promote cell-fate reproduced germ cells and somatic cells . Our results will provide the basis for a novel development strategy in the cellular reproduction of germ cells . . . . Reprogramming is at the heart of development . . . However , very little is known about the molecular mechanisms promoting or inhibiting cell-fate reproduction in intact organisms . This is surprising , since GLP-1/Notch is best known for maintaining undifferentiated germline stem cells/progenitors . We propose that GLP-1/Notch promotes reprograming by activating specific genes , silenced 

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.


[Summary] SummacConv score: 0.251


In [ ]:
#falconsi
from transformers import pipeline

summarizer = pipeline("summarization", model="Falconsai/medical_summarization")
sum = summarizer(input, max_length=250, min_length=80, do_sample=False)
compute_rouge(sum)
compute_bertscore(sum)
calculate_readability_scores(sum)
compute_summac(sum)

config.json:   0%|          | 0.00/1.50k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.37k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

...

      Metric  Precision    Recall  FMeasure
0     rouge1   0.475410  0.083573  0.142157
1     rouge2   0.016667  0.002890  0.004926
2     rougeL   0.213115  0.037464  0.063725
3  rougeLsum   0.213115  0.037464  0.063725


/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for bertscore contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/bertscore/bertscore.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to

      Metric     Score
0  precision  0.821480
1     recall  0.802642
2         f1  0.811952
we describe that such reprograming is facilitated by GLP-1/Notch signaling pathway . this is surprising , since this pathway is best known for maintaining undifferentiated germline stem cells/progenitors . through a combination of genetics , tissue-specific transcriptome analysis , and functional studies of candidate genes , we uncovered a possible explanation for this unexpected role of a gene - mediated gene gene .
Flesch-Kincaid Grade Level: 15.1
Dale-Chall Readability Score: 12.87
<All keys matched successfully>


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.


[Summary] SummacConv score: 0.717


In [ ]:
#finetuned falconsi
from transformers import pipeline
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
# Load the trained model and tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/Dissertation1/model_falconsi")
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/Dissertation1/tokenizer_falconsi")

# Create the summarization pipeline
summarizer = pipeline("summarization", model=model, tokenizer=tokenizer)
sum = summarizer(input, max_length=250, min_length=80, do_sample=False)

compute_rouge(sum)
compute_bertscore(sum)
calculate_readability_scores(sum)
compute_summac(sum)

/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


      Metric  Precision    Recall  FMeasure
0     rouge1   0.438356  0.092219  0.152381
1     rouge2   0.027778  0.005780  0.009569
2     rougeL   0.219178  0.046110  0.076190
3  rougeLsum   0.219178  0.046110  0.076190


/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for bertscore contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/bertscore/bertscore.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to

      Metric     Score
0  precision  0.818920
1     recall  0.804046
2         f1  0.811415
we describe that such reprograming is facilitated by GLP-1/Notch signaling pathway . this is surprising , since this pathway is best known for maintaining undifferentiated germline stem cells/progenitors . through a combination of genetics , tissue-specific transcriptome analysis , and functional studies of candidate genes , we uncovered a possible explanation for this unexpected role of a gene . we propose that the conserved histone demethylase UTX-1 as a crucial target facilitating reprogramming .
Flesch-Kincaid Grade Level: 14.4
Dale-Chall Readability Score: 13.31
<All keys matched successfully>


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.


[Summary] SummacConv score: 0.686


In [ ]:
from transformers import pipeline
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
# Load the trained model and tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/Dissertation/modelT5")
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/Dissertation/tokenizerT5")

# Initialize the pipeline
text_generator = pipeline("text2text-generation", model=model, tokenizer=tokenizer, device=-1)

# Use the pipeline to generate text
prompt =  'summarize in simple terms:'+ input
sum = text_generator(prompt, max_length=250, min_length=180, do_sample=True, temperature=0.7)

print(sum)
compute_rouge(sum)
compute_bertscore(sum)
calculate_readability_scores(sum)
compute_summac(sum)

[{'generated_text': 'a signaling pathway called the GLP-1/Notch pathway facilitated cell-fate reprograming in the germline . this is a key mechanism enabling cell-fate reprograming in intact organisms . in the C . elegans germline , reprograming germ cells involves chromatin perturbation . these findings have wide implications , ranging from development to diseases associating with germline stem cells . the pathway is best known for maintaining undifferentiated germline stem , . , , , - l - - -  n -         - , - an n .  . - - - s s    , p  '}]


/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


      Metric  Precision    Recall  FMeasure
0     rouge1   0.493151  0.103746  0.171429
1     rouge2   0.069444  0.014451  0.023923
2     rougeL   0.273973  0.057637  0.095238
3  rougeLsum   0.273973  0.057637  0.095238


/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for bertscore contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/bertscore/bertscore.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to

      Metric     Score
0  precision  0.794168
1     recall  0.802996
2         f1  0.798558
a signaling pathway called the GLP-1/Notch pathway facilitated cell-fate reprograming in the germline . this is a key mechanism enabling cell-fate reprograming in intact organisms . in the C . elegans germline , reprograming germ cells involves chromatin perturbation . these findings have wide implications , ranging from development to diseases associating with germline stem cells . the pathway is best known for maintaining undifferentiated germline stem , . , , , - l - - -  n -         - , - an n .  . - - - s s    , p  
Flesch-Kincaid Grade Level: 8.9
Dale-Chall Readability Score: 11.83
<All keys matched successfully>


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


[Summary] SummacConv score: 0.297


In [ ]:
from transformers import pipeline
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
# Load the trained model and tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/Dissertation/modelT5")
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/Dissertation/tokenizerT5")

# Create the summarization pipeline
summarizer = pipeline("summarization", model=model, tokenizer=tokenizer)
sum = summarizer(input, max_length=250, min_length=80, do_sample=False)

compute_rouge(sum)
compute_bertscore(sum)
calculate_readability_scores(sum)
compute_summac(sum)

/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


      Metric  Precision    Recall  FMeasure
0     rouge1   0.456140  0.074928  0.128713
1     rouge2   0.017857  0.002890  0.004975
2     rougeL   0.298246  0.048991  0.084158
3  rougeLsum   0.298246  0.048991  0.084158


/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for bertscore contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/bertscore/bertscore.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to

      Metric     Score
0  precision  0.826738
1     recall  0.803364
2         f1  0.814883
cell-fate reprograming is at the heart of development , yet very little is known about the molecular mechanisms promoting or inhibiting reprogramming in intact organisms . in the C . elegans germline , reprogramping germ cells into somatic cells requires chromatin perturbation . this is surprising , since this pathway is best known for maintaining undifferentiated germline stem cells/progenitors .
Flesch-Kincaid Grade Level: 12.0
Dale-Chall Readability Score: 11.74
<All keys matched successfully>


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


[Summary] SummacConv score: 0.877


In [ ]:
from transformers import pipeline
from transformers import AutoModelForCausalLM, AutoTokenizer
# Load the trained model and tokenizer
model = AutoModelForCausalLM.from_pretrained("/content/drive/MyDrive/Dissertation1/model_biogpt")
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/Dissertation1/tokenizer_biogpt")

# Initialize the pipeline
text_generator = pipeline("text-generation", model=model, tokenizer=tokenizer, device=-1)

# Use the pipeline to generate text
prompt =  'summarize in simple terms:'+ input
sum = text_generator(prompt, max_length=250, min_length=180, do_sample=True, temperature=0.7)

print(sum)
compute_rouge(sum)
compute_bertscore(sum)
calculate_readability_scores(sum)
compute_summac(sum)

[{'generated_text': 'summarize in simple terms:Cell-fate reprograming is at the heart of development , yet very little is known about the molecular mechanisms promoting or inhibiting reprograming in intact organisms . In the C . elegans germline , reprograming germ cells into somatic cells requires chromatin perturbation . Here , we describe that such reprograming is facilitated by GLP-1/Notch signaling pathway . This is surprising , since this pathway is best known for maintaining undifferentiated germline stem cells/progenitors . Through a combination of genetics , tissue-specific transcriptome analysis , and functional studies of candidate genes , we uncovered a possible explanation for this unexpected role of GLP-1/Notch . We propose that GLP-1/Notch promotes reprograming by activating specific genes , silenced by the Polycomb repressive complex 2 ( PRC2 ) , and identify the conserved histone demethylase UTX-1 as a crucial GLP-1/Notch target facilitating reprograming . These findin

<ipython-input-12-f100f6e456a4>:5: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge = load_metric("rouge")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


      Metric  Precision    Recall  FMeasure
0     rouge1   0.423077  0.190202  0.262425
1     rouge2   0.032258  0.014451  0.019960
2     rougeL   0.198718  0.089337  0.123260
3  rougeLsum   0.198718  0.089337  0.123260


/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for bertscore contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/bertscore/bertscore.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to

      Metric     Score
0  precision  0.820723
1     recall  0.824936
2         f1  0.822824
summarize in simple terms:Cell-fate reprograming is at the heart of development , yet very little is known about the molecular mechanisms promoting or inhibiting reprograming in intact organisms . In the C . elegans germline , reprograming germ cells into somatic cells requires chromatin perturbation . Here , we describe that such reprograming is facilitated by GLP-1/Notch signaling pathway . This is surprising , since this pathway is best known for maintaining undifferentiated germline stem cells/progenitors . Through a combination of genetics , tissue-specific transcriptome analysis , and functional studies of candidate genes , we uncovered a possible explanation for this unexpected role of GLP-1/Notch . We propose that GLP-1/Notch promotes reprograming by activating specific genes , silenced by the Polycomb repressive complex 2 ( PRC2 ) , and identify the conserved histone demethylase UTX-1 a

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


[Summary] SummacConv score: 0.743


/usr/local/lib/python3.10/dist-packages/summac/model_summac.py:298: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  histograms = torch.FloatTensor(histograms).to(self.device)


In [ ]:
from transformers import pipeline
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
# Load the trained model and tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/Dissertation1/model_ft5l")
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/Dissertation1/tokenizer_ft5l")

# Initialize the pipeline
text_generator = pipeline("text2text-generation", model=model, tokenizer=tokenizer)

# Use the pipeline to generate text
prompt =  'summarize in simple terms:'+ input
sum = text_generator(prompt, max_length=250, min_length=180, do_sample=True, temperature=0.7)

print(sum)
compute_rouge(sum)
compute_bertscore(sum)
calculate_readability_scores(sum)
compute_summac(sum)

[{'generated_text': 'Our report demonstrates that GLP-1/Notch promotes somatic cell reprogramming by activating conserved histone demethylase UTX-1 . This finding is surprising , since GLP-1/Notch is best known for maintaining undifferentiated germline stem cells/progenitors. The results provide novel insights into the molecular mechanisms promoting and inhibiting somatic cell reprogramming. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . '}]


/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


      Metric  Precision    Recall  FMeasure
0     rouge1   0.301887  0.046110  0.080000
1     rouge2   0.019231  0.002890  0.005025
2     rougeL   0.245283  0.037464  0.065000
3  rougeLsum   0.245283  0.037464  0.065000


/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for bertscore contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/bertscore/bertscore.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to

      Metric     Score
0  precision  0.755893
1     recall  0.797834
2         f1  0.776298
Our report demonstrates that GLP-1/Notch promotes somatic cell reprogramming by activating conserved histone demethylase UTX-1 . This finding is surprising , since GLP-1/Notch is best known for maintaining undifferentiated germline stem cells/progenitors. The results provide novel insights into the molecular mechanisms promoting and inhibiting somatic cell reprogramming. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
Flesch-Kincaid Grade Level: 15.0
Dale-Chall Readability Score: 13.85
<All keys matched successfully>


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


[Summary] SummacConv score: 0.223
